**Objetivo:** Leer y estandarizar los datasets (SIES, DEMRE, IVM, georreferenciación, etc.)

- Cargar datos de todas las fuentes
- Verificar formatos y columnas clave
- Establecer claves de unión (`MRUN`, `RBD`, códigos de institución)
- Guardar versiones limpias en `data/processed/` (opcional)

In [1]:
import pandas as pd
from pathlib import Path
import utils

# Configuraciones generales
pd.set_option("display.max_columns", None)

In [2]:
# Lectura 
list( Path("../data/raw/2021").glob('*'))

[WindowsPath('../data/raw/2021/20230802_Matrícula_Ed_Superior_2021_PUBL_MRUN.csv'),
 WindowsPath('../data/raw/2021/A_INSCRITOS_PUNTAJES_PDT_2021_PUB_MRUN.csv'),
 WindowsPath('../data/raw/2021/establecimientos'),
 WindowsPath('../data/raw/2021/inmuebles_ies'),
 WindowsPath('../data/raw/2021/IVM_Establecimientos_2021.xlsx')]

In [3]:
# Ruta base
path_base = Path('../data/raw')
year = 2021

# Obtener paths del año
path_year = utils.obtener_paths_anio(path_base, year)
path_year

{'matricula': WindowsPath('../data/raw/2021/20230802_Matrícula_Ed_Superior_2021_PUBL_MRUN.csv'),
 'puntajes': WindowsPath('../data/raw/2021/A_INSCRITOS_PUNTAJES_PDT_2021_PUB_MRUN.csv'),
 'ivm': WindowsPath('../data/raw/2021/IVM_Establecimientos_2021.xlsx'),
 'establecimientos': WindowsPath('../data/raw/2021/establecimientos/layer_establecimientos_educacion_escolar_20220309024120.shp'),
 'inmuebles_ies': WindowsPath('../data/raw/2021/inmuebles_ies/layer_establecimientos_de_educacion_superior_20220309024111.shp')}

In [4]:
# Leer conjuntos individuales
set_a = utils.leer_conjunto_a_desde_path(path_year['matricula'], utils.region_dict)
set_b = utils.leer_conjunto_b_desde_path(path_year['puntajes'], utils.region_dict)
set_c = utils.leer_conjunto_c_desde_path(path_year['establecimientos'])
set_d = utils.leer_conjunto_d_desde_path(path_year['inmuebles_ies'], utils.region_dict)
set_e = utils.leer_conjunto_e_desde_path(path_year['ivm'])

In [6]:
# Filtrar A1
resultados_a = utils.generar_conjuntos_filtrados(set_a, year)
set_a0 = resultados_a['A0']
set_aa = resultados_a['A']
set_a1 = resultados_a['A1']


In [7]:
# Carpeta de salida
output_path = Path(f'../data/clean/{year}')
output_path.mkdir(parents=True, exist_ok=True)

# Guardar conjuntos individuales
set_a.to_csv(output_path / 'set_a.csv', index=False)
set_a0.to_csv(output_path / 'set_a0.csv', index=False)
set_aa.to_csv(output_path / 'set_aa.csv', index=False)
set_a1.to_csv(output_path / 'set_a1.csv', index=False)
set_b.to_csv(output_path / 'set_b.csv', index=False)
set_c.to_csv(output_path / 'set_c.csv', index=False)
set_d.to_csv(output_path / 'set_d.csv', index=False)
set_e.to_csv(output_path / 'set_e.csv', index=False)

In [8]:
# Generar y guardar conjuntos integrados
utils.generar_conjuntos_abcde(
    set_a=set_a1,
    set_b=set_b,
    set_c=set_c,
    set_d=set_d,
    set_e=set_e,
    output_path=output_path
)


Filling coordinates for 50976 missing entries...
Successfully filled 33634 missing coordinates
Remaining NA values: 17342
Cantidad de observaciones: 77366


**Objetivo:** Explorar y analizar los datos limpios generados en la etapa de lectura.

- Revisar estructura general y dimensiones de los datasets  
- Analizar variables clave y su distribución  
- Identificar outliers, datos faltantes y relaciones relevantes  
- Generar visualizaciones exploratorias para apoyar el análisis  
